# Parse PATH monthly ridership PDF
```bash
papermill -p year $year [-p last_month $last_month] monthly.ipynb out/monthly-$year.ipynb
```

In [ ]:
from utz import *
import json
from tabula import read_pdf

In [ ]:
n = now()
cur_year = n.year
cur_month = n.month
cur_year, cur_month

Papermill parameters:

In [ ]:
year = None
last_month = None
template = None

In [ ]:
if year is None:
    year = cur_year

if last_month is None:
    if year == cur_year:
        last_moonth = 1
    else:
        last_month = 12

if template is None:
    template = 'PATH-Monthly-Ridership-Report.json'

In [ ]:
pdf = f'data/{year}-PATH-Monthly-Ridership-Report.pdf'
pdf

In [ ]:
with open(f'templates/{template}', 'r') as f:
    rects = json.load(f)
rects

In [ ]:
area = [ [ r[k] for k in [ 'y1', 'x1', 'y2', 'x2' ] ] for r in rects ]
area

In [ ]:
tables = {
    month: read_pdf(
        pdf,
        pages=month,
        area=area,
        pandas_options={'header': None},
        stream=True,
    ) 
    for month in range(1, last_month + 1)
}

tables

In [ ]:
cols1 = [ 'station', 'total', 'avg weekday', 'avg sat', 'avg sun', 'avg holiday' ]
cols2 = [ 'station', 'avg daily', 'total weekday', 'total sat', 'total sun', 'total holiday' ]

In [ ]:
for k, v in tables.items():
    n = len(v)
    msg = f'Pg {k}: {n} tables'
    if n == 4:
        print(msg)
    else:
        err(msg)
len(tables)

In [ ]:
avgs = pd.concat([
    df.assign(date=to_dt(f'{year}-{month}'))
    for month, dfs in tables.items()
    for df in dfs[:2]
])
avgs

In [ ]:
avgs.columns = cols1 + ['month']
avgs = avgs.assign(**{
    k: avgs[k].astype(str).str.replace(',', '').astype(int)
    for k in cols1[1:]
})
avgs

In [ ]:
avgs.isna().sum(axis=0)

In [ ]:
avgs.dtypes

In [ ]:
sums = pd.concat([
    df.assign(date=to_dt(f'{year}-{month}'))
    for month, dfs in tables.items()
    for df in dfs[2:]
])
sums.columns = cols2 + ['month']
sums = sums.assign(**{
    k: sums[k].astype(str).str.replace(',', '').astype(int)
    for k in cols2[1:]
})
sums

In [ ]:
sums.isna().sum(axis=0)

In [ ]:
sums.dtypes

In [ ]:
df = sxs(
    avgs.set_index(['month', 'station']),
    sums.set_index(['month', 'station']),
)
df = df[[cols2[1]] + cols1[2:] + [cols1[1]] + cols2[2:]]
df

In [ ]:
path = f'data/{year}.pqt'
df.to_parquet(path)

In [ ]:
import plotly.express as px
from IPython.display import Image

In [ ]:
fig = px.bar(
    avgs[~avgs.station.str.contains('TOTAL')],
    x='month', y='avg weekday', color='station',
)
Image(fig.to_image(width=1200, height=600))